In [ ]:
%%bash
mkdir /python/output

In [1]:
# general
import os
import random
import csv

# wolfram alpha
import wolframalpha

# images
import urllib.request

# email
import smtplib, ssl
from email.mime.text import MIMEText
from email.mime.multipart import MIMEMultipart
from email.mime.image import MIMEImage

In [2]:
# get list from csv

with open('/python/input/emails.csv', newline='') as f:
    reader = csv.reader(f)
    receiver_email = [row[0] for row in reader]
    
receiver_email

['ccarbonell228@gmail.com']

Source:<br>
https://www.geeksforgeeks.org/python-create-a-simple-assistant-using-wolfram-alpha-api/

In [3]:
gbl_app_id = '2H79RA-9HVG6K889K' # improvement: get from secrets.py
gbl_str_path_output = "/python/output/" # end with last /

In [ ]:
# randomize
gbl_str_gender = random.choice(['boys','girls'])
gbl_int_rank = random.randint(1,5000)

In [4]:
# wc = wolframalpha.Client(gbl_app_id)
wc = wolframalpha.Client(gbl_app_id)

In [108]:
# testing
# question = " ".join(["#" + str(gbl_int_rank), gbl_str_gender, "name"]) # ie "#1 boys name"
# res = wc.query(question)
res = wc.query("#3547 boys name")
# res = wc.query("origin of boys name Chris")

In [13]:
# subpods are just dicts:
# "All objects returned are dictionary subclasses, so to find out which attributes Wolfram|Alpha has supplied, simply invoke .keys() on the object. Attributes formed from XML attributes can be accessed with or without their “@” prefix (added by xmltodict)."
for pod in res.pods:
    for sub in pod.subpods:
        print(sub.keys())

dict_keys(['@title', 'img', 'plaintext'])
dict_keys(['@title', 'microsources', 'datasources', 'img', 'plaintext'])


In [22]:
res['@success'] == 'false'
print(res['@success'])

true


In [104]:
ls_plaintext = pod['subpod']['plaintext'].split('\n')
ls_plaintext

['⋮ | | | ',
 '611 | Abdiel | 439 people/yr | ',
 '612 | Hamza | 438 people/yr | ',
 '613 | Chris | 437 people/yr | ',
 '614 | Alfredo | 437 people/yr | ',
 '615 | Finnley | 436 people/yr | ',
 '⋮']

In [109]:
list(res.pods)[1]

{'@title': 'Result',
 '@scanner': 'Ordinal',
 '@id': 'Result',
 '@position': '200',
 '@error': 'false',
 '@numsubpods': '1',
 '@primary': 'true',
 'subpod': {'@title': '',
  'datasources': {'datasource': 'USSSABabyNames'},
  'img': {'@src': 'https://www5a.wolframalpha.com/Calculate/MSP/MSP781317gaai7a8iged26c00004d5ab26gaeeg07bh?MSPStoreType=image/gif&s=40',
   '@alt': 'Keane (31 people per year)',
   '@title': 'Keane (31 people per year)',
   '@width': '133',
   '@height': '38',
   '@type': 'Default',
   '@themes': '1,2,3,4,5,6,7,8,9,10,11,12',
   '@colorinvertable': 'true'},
  'plaintext': 'Keane\n(31 people per year)'},
 'expressiontypes': {'@count': '1', 'expressiontype': {'@name': 'Grid'}}}

In [89]:
pod['subpod']['plaintext'].split('\n')[-1]

'⋮'

In [37]:
next(res.pods)

{'@title': 'Input interpretation',
 '@scanner': 'Identity',
 '@id': 'Input',
 '@position': '100',
 '@error': 'false',
 '@numsubpods': '1',
 'subpod': {'@title': '',
  'img': {'@src': 'https://www5b.wolframalpha.com/Calculate/MSP/MSP168140aef83bgg0g42g00004c9gb577gef3bf71?MSPStoreType=image/gif&s=51',
   '@alt': 'Chris (male given name) | etymology',
   '@title': 'Chris (male given name) | etymology',
   '@width': '261',
   '@height': '23',
   '@type': 'Grid',
   '@themes': '1,2,3,4,5,6,7,8,9,10,11,12',
   '@colorinvertable': 'true'},
  'plaintext': 'Chris (male given name) | etymology'},
 'expressiontypes': {'@count': '1', 'expressiontype': {'@name': 'Grid'}}}

In [50]:
# locations of info
int_pod = 0
for pod in res.pods:
    int_sub = 0
    for sub in pod.subpods:
        print(int_pod, int_sub, sub['plaintext'])
            
        int_sub = int_sub + 1
        print()
    
    int_pod = int_pod + 1
    
# no pods have more than one sub

0 0 613th US male given names | by current naming frequency

1 0 Chris
(437 people per year)

2 0 rank | 613th
fraction | 1 in 4120 people (0.024%)
number | 437 people per year
(US data based on 2018 births and other SSA registrations in the US)

3 0 None

4 0 expected total number alive today | 126812 people
expected population fraction | 1 in 2074 people (0.048%)
expected rank | 413th
most common age | 49 years
(using standard US mortality data)

5 0 None

6 0 Christian | Christ'o | Christopher | Kester | Kitappa | Tenny | Tiny | Topher | Xian | Xph'r | Xpr | Xtianus | Xtopherus

7 0 Chris Hemsworth (actor) (born 1983) (age: 37 years)
Chris Jericho (wrestler) (born 1970) (age: 49 years)
Chris Pratt (actor) (born 1979) (age: 41 years)
Chris Pine (actor) (born 1980) (age: 40 years)
Chris Martin (singer/songwriter, etc.) (born 1977)
Chris Rock (comedian) (born 1965) (age: 55 years)
Chris Benoit (wrestler) (1967 to 2007)
Chris Farley (comedian) (1964 to 1997)
Chris Tucker (comedian) (197

In [14]:
# locations of info
int_pod = 0
for pod in res.pods:
    int_sub = 0
    for sub in pod.subpods:
        print(int_pod, int_sub, sub['plaintext'])
            
        int_sub = int_sub + 1
        print()
    
    int_pod = int_pod + 1
    
# no pods have more than one sub

0 0 Chris (male given name) | etymology

1 0 Short form of Christopher, Christian and other names that begin with Chris



In [ ]:
# save raw data
dict_raw = {}
int_pod = 0
for pod in res.pods:
    dict_raw[int_pod] = {}
    sub = next(pod.subpods) # there's only one
    if sub['plaintext'] is not None:
        dict_raw[int_pod] = sub['plaintext'].split('\n')
    else:
        dict_raw[int_pod] = {'@src': sub['img']['@src'], '@alt': sub['img']['@alt']}
    
    int_pod = int_pod + 1
    
dict_raw

In [ ]:
# save raw data
dict_raw = {}
int_pod = 0
for pod in res.pods:
    dict_raw[int_pod] = {}
    sub = next(pod.subpods) # there's only one
    if sub['plaintext'] is not None:
        dict_raw[int_pod] = sub['plaintext'].split('\n')
    else:
        dict_raw[int_pod] = {'@src': sub['img']['@src'], '@alt': sub['img']['@alt']}
    
    int_pod = int_pod + 1
    
dict_raw

In [ ]:
# downlaod images

str_path_img_history = os.path.join(gbl_str_path_output, str(gbl_int_rank).zfill(5) + " " + dict_raw[1][0] + "-history.jpg")
str_path_img_age = os.path.join(gbl_str_path_output, str(gbl_int_rank).zfill(5) + " " + dict_raw[1][0] + "-age.jpg")

urllib.request.urlretrieve(dict_raw[3]['@src'], str_path_img_history)
urllib.request.urlretrieve(dict_raw[5]['@src'], str_path_img_age)

In [ ]:
# send email
# https://realpython.com/python-send-email/

sender_email = "carbobot3030@gmail.com"
# receiver_email = ["ccarbonell228@gmail.com", "katiemoore1010@gmail.com"]
receiver_email = ["ccarbonell228@gmail.com"] # improvement: get from shared volume so we can take her off

message = MIMEMultipart("alternative")
message["Subject"] = "Baby Name: " + dict_raw[1][0]
message["From"] = sender_email
message["To"] = ", ".join(receiver_email)

text = """
""" + dict_raw[1][0] + """
""" + dict_raw[8][0] + """
"""

html = """\
<html>
  <head></head>
  <body>
    <h1>""" + dict_raw[1][0] + """</h1>
    <b style='color:blue'><i>""" + dict_raw[8][0] + """</i></b>
    
    <h2>Random Stats</h2>
    <ul>
        <li>""" + dict_raw[2][0].split("|")[1].strip() + """ most popular</li>
        <li>""" + dict_raw[2][1].split("|")[1].strip() + """</li>
        <li>""" + dict_raw[2][2].split("|")[1].strip() + """</li>
        <li>""" + dict_raw[4][0].replace("|","=") + """</li>
        <li>""" + dict_raw[4][3].replace("|","=") + """</li>
    </ul>
    
    <h2>""" + dict_raw[3]['@alt'] + """</h2>
    <img src="cid:image1">
    
    <h2>""" + dict_raw[5]['@alt'] + """</h2>
    <img src="cid:image2">
    
    <h2>Famous Examples</h2>
    <ul>
    """ + "\n".join(["<li>" + str_example + "</li>" for str_example in dict_raw[7]]) + """
    </ul>
    
    <h2>Sources</h2>
    <ul>
        <li>""" + dict_raw[2][3][1:-1] + """</li>
        <li>""" + dict_raw[4][4][1:-1] + """</li>
    </ul>
  </body>
</html>
"""

# Record the MIME types of both parts - text/plain and text/html.
part1 = MIMEText(text, 'plain')
part2 = MIMEText(html, 'html')

# Attach parts into message container.
# According to RFC 2046, the last part of a multipart message, in this case
# the HTML message, is best and preferred.
message.attach(part1)
message.attach(part2)

# download, encode, and attach image

fp = open(str_path_img_history, 'rb')
msgImage_history = MIMEImage(fp.read())
fp.close()

fp = open(str_path_img_age, 'rb')
msgImage_age = MIMEImage(fp.read())
fp.close()

# Define the image's ID as referenced above

msgImage_history.add_header('Content-ID', '<image1>')
message.attach(msgImage_history)

msgImage_age.add_header('Content-ID', '<image2>')
message.attach(msgImage_age)

# connect to gmail
# for this to work, we need to turn ON less secure app access

port = 465  # For SSL
password = "delthefunkyhomosapien" # improvement: get from secrets

# Create a secure SSL context
context = ssl.create_default_context()
with smtplib.SMTP_SSL("smtp.gmail.com", 465, context=context) as server:
    server.login(sender_email, password)
    server.sendmail(
        sender_email, receiver_email, message.as_string()
    )